In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.models import Word2Vec
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
import lightgbm as lgbm
import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

# Загрузка данных
train = pd.read_csv('train_spam.csv')
test = pd.read_csv('test_spam.csv')

# посмотрим несколько подходов к задаче:

#### 1. Первый подход - с помощью Tf Idf векторизации и логистической регрессии
#### 2. Второй подход - с помощью Tf Idf векторизации и бустингов
#### 3. Третий подход - с помощью word2vec и логистической регрессии
#### 4. Четвертый подход - с помощью word2vec и бустингов
#### 5. Пятый подход - с помощью RNN (В отдельном ноутбуке (RNN.ipynb))

#### все подходы будут оцениваться с помощью метрики accuracy используя кросс-валидацию

# 1) Первый подход - с помощью Tf Idf векторизации и логистической регрессии

In [43]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train['text'])
X_test = vectorizer.transform(test['text'])
# заменим 'ham' на 0, 'spam' на 1
y_train = train['text_type'].map({'ham': 0, 'spam': 1})

In [15]:
# используем кросс-валидацию для оценки качества логистической регрессии
logreg = LogisticRegression()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
log_values = cross_val_score(logreg, X_train, y_train, cv=cv, scoring='roc_auc')

/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [16]:
print('LogisticRegression mean ROC-AUC: ', log_values.mean())

LogisticRegression mean ROC-AUC:  0.9771549754264998


# 2) Второй подход - с помощью Tf Idf векторизации и бустингов

In [44]:
#1) lgbm
lgm_params = {
    'objective': 'binary',
    'metric': 'roc_auc',
    'boosting_type': 'gbdt',
    'n_estimators': 300,
    'learning_rate': 0.1,
    'verbose': -1
}

lgbm_clf = lgbm.LGBMClassifier(**lgm_params)
lgbm_values = cross_val_score(lgbm_clf, X_train, y_train, cv=cv, scoring='roc_auc')


/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [45]:
print('LGBM mean ROC-AUC: ', lgbm_values.mean())

LGBM mean ROC-AUC:  0.984065393795418


In [46]:
# запишем предсказания в файл (лучший результат)
lgbm_clf.fit(X_train, y_train)
test['score'] = lgbm_clf.predict_proba(X_test)[:, 1]
test[['text', 'score']].to_csv('best.csv', index=False)

/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [22]:
#2) catboost
cat_params = {
    'iterations': 300,
    'learning_rate': 0.1,
    'verbose': False
}

cat_clf = CatBoostClassifier(**cat_params)
cat_values = cross_val_score(cat_clf, X_train, y_train, cv=cv, scoring='roc_auc')

/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [23]:
print('CatBoost mean ROC-AUC: ', cat_values.mean())

CatBoost mean ROC-AUC:  0.9790320186063697


In [27]:
#3) xgboost
xgb_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'n_estimators': 300,
    'learning_rate': 0.1,
    'verbosity': 0
}

xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_values = cross_val_score(xgb_clf, X_train, y_train, cv=cv, scoring='roc_auc')

/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will

In [28]:
print('XGBoost mean ROC-AUC: ', xgb_values.mean())

XGBoost mean ROC-AUC:  0.9803024456601233


# 3) Третий подход - с помощью word2vec и логистической регрессии>

In [34]:
# используем word2vec
class MeanEmbeddingVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = word2vec.vector_size
    
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
word2vec = Word2Vec(train['text'].apply(str.split), min_count=1)
mean_embedding_vectorizer = MeanEmbeddingVectorizer(word2vec.wv)



# используем кросс-валидацию для оценки качества логистической регрессии




In [36]:
X_train = mean_embedding_vectorizer.fit_transform(train['text'], y_train)
X_test = mean_embedding_vectorizer.transform(test['text'])

In [37]:
logreg = LogisticRegression()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
log_values = cross_val_score(logreg, X_train, y_train, cv=cv, scoring='roc_auc')

/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/andrey/anaconda3/envs/environment_2023_24yml/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is dep

In [38]:
print('LogisticRegression mean ROC-AUC: ', log_values.mean())

LogisticRegression mean ROC-AUC:  0.7748145365721275


## заметим что word2vec сильно хуже tf idf векторизации (логично, ведь в word2vec учитывается только семантика слов, а в tf idf - еще и частота встречаемости слова в документе, что важно для определения спама), поэтому смотреть на бустинги с word2vec не будем

# Вывод: лучший результат показала Tf Idf векторизация и lgbm (accuracy = 0.984) запись в файле best.csv
#### Очень хорошие тексты для обучения и очень мало данных, поэтому задача почти идеально решилась простыми методами (Tf Idf + логистическая регрессия или бустинги). Возможно, если бы было больше данных, то RNN показала бы себя лучше (но в данном случае дала score = 0.95). Word2vec показал себя хуже, чем Tf Idf (описано выше)

##### Замечание: в данной задаче можно было бы пооптюнить гиперпараметры, но в данном случае этого делать не стал, так как результат уже очень хороший, также для улучшения моделей можно было бы добавить еще фичи (например длину текста, количество слов в тексте, наличия смайликов и т.д.)